# Text-based Graph Convolutional Network


Sources:

- https://arxiv.org/pdf/1809.05679.pdf  
- https://arxiv.org/pdf/1609.02907.pdf  
- https://github.com/plkmo/Bible_Text_GCN   

In [ ]:
import pandas as pd 
import numpy as np
import keras
from sklearn.feature_extraction.text import TfidfVectorizer 
import pickle
import os
from collections import OrderedDict
import networkx as nx

from tqdm import tqdm
from itertools import combinations
import math

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

%matplotlib inline


def nCr(n, r):
    f = math.factorial
    return int(f(n)/(f(r)*f(n-r)))


Using TensorFlow backend.


In [3]:
# FILE PATH adjusted for local
def save_as_pickle(filename, data):
    completeName = os.path.join("../data", filename)
    print(completeName)
    with open(completeName, 'wb') as output:
        pickle.dump(data, output)
        
def load_pickle(filename):
    completeName = os.path.join("../data", filename)
    with open(completeName, 'rb') as pkl_file:
        data = pickle.load(pkl_file)
    return data


def load_data(load_from_disc, debug, test_size = None, train_size = None, vocab_size = None):
    """
    load_from_disc: load data from s3 or from own disk; the later ther must be stored priorly, 
    debug: defines if the train and test data filtered, 
    test_size: size of the test data  (Debug = True)
    train_size = size of the train data  (Debug = True)
    vocab_size = size of the vocabolary (load_from_disc = False)
    """
    if load_from_disc:
        docs = load_pickle("data_joined.pkl")
        y_train = load_pickle("y_train.pkl")
        y_test = load_pickle("y_test.plk")

        # Index for semi supervised test data 
        idx_test = [i for i in range(len(y_train), len(y_train) + len(y_test) )]
        idx_train = [i for i in range(len(y_train))]

        if debug:
            # DEBUG!!!
            idx_train = idx_train[:train_size]
            idx_test = idx_test[:test_size] 

            X_train = [docs[i] for i in idx_train]
            X_test = [docs[i] for i in idx_test]
            docs = np.append(X_train, X_test)

            y_train = [y_train[i] for i in [idx_train]]
            y_test = y_test[:test_size]


    else:
        vocab_size = vocab_size
        imdb = keras.datasets.imdb

        (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocab_size)

        if debug:
            # DEBUG!!!
            X_train = X_train[:train_size]
            y_train = y_train[:train_size]

            X_test = X_test[:5]
            y_test = y_test[:5]

        # Decode imdb index to words
        imdb_word_index = imdb.get_word_index()
        imdb_word_index = {k: (v + 3) for k,v in imdb_word_index.items()}
        imdb_word_index["<PAD>"] = 0
        imdb_word_index["<START>"] = 1
        imdb_word_index["<UNK>"] = 2
        imdb_word_index["<UNUSED>"] = 3

        imdb_index_word = {idx: value for value, idx in imdb_word_index.items()}

        def decode_text(txt):
            return ' '.join([imdb_index_word.get(i, '?') for i in txt])


        # Create data for semi-supervised classifier
        data = np.append(X_train, X_test)
        docs = list(map(lambda i: decode_text(i), data )) # decode idx to text


        # Index for semi supervised test data 
        idx_test = [i for i in range(len(y_train), len(y_train) + len(y_test) )]
        idx_train = [i for i in range(len(y_train))]

        # Store results
        save_as_pickle("data_joined.pkl", docs)
        save_as_pickle("y_train.pkl", y_train)
        save_as_pickle("y_test.plk", y_test)
    

    return docs, y_train, y_test

# Fetch Data



In [ ]:
docs, y_train, y_test = load_data(load_from_disc = True, debug = True, train_size = 10000, test_size = 10000, vocab_size = 5000)

# Calculate the Graph Edges
****

<img src="weights_formula.png" alt="drawing" width="500">


**PMI**  

* PMI is the *Point-wise Mutual Information* between pairs of co-occurring words over a sliding window $\#W$ that we fix to be of 10-words length
* $\#W(i)$ is the number of sliding windows in a corpus that contain word $i$  
* $\#W(i,j)$ is the number of sliding windows that contain both word $i$ and $j$  
* $\#W$ is the total number of sliding windows in the corpus


**TF-iDF** 
For the word-document relationship in the graph the author proposed the tf-idf method. As this is the easer information and a well known method we will start here.



## TF-iDF
***

Our target is to create a data frame with the dimension [num docs x vocabulary size] containing the TF-iDF score for each word and document.


In [5]:
tfidf = TfidfVectorizer(input = "content")
tfidf_vector = tfidf.fit(docs) 
df_tfidf = tfidf_vector.transform(docs)
df_tfidf = df_tfidf.toarray()

# Transform to a data frame
vocab = tfidf.get_feature_names(); vocab = np.array(vocab)
df_tfidf = pd.DataFrame(df_tfidf, columns=vocab)
df_tfidf

,10,about,acting,actors,actually,after,again,all,also,an,...,why,will,with,work,world,would,years,you,young,your
0,0.000000,0.000000,0.000000,0.0,0.000000,0.032836,0.000000,0.065817,0.031299,0.022770,...,0.00000,0.000000,0.036056,0.0,0.000000,0.028304,0.000000,0.087607,0.000000,0.000000
1,0.000000,0.000000,0.033705,0.0,0.039557,0.000000,0.040497,0.021971,0.000000,0.022803,...,0.00000,0.000000,0.018055,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.049408,0.0,0.057986,0.000000,0.000000,0.000000,0.000000,0.066854,...,0.00000,0.000000,0.026466,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.053613
3,0.000000,0.007552,0.000000,0.0,0.000000,0.009977,0.000000,0.006666,0.019020,0.020756,...,0.00000,0.009604,0.032867,0.0,0.012950,0.000000,0.023753,0.033274,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.0,0.047667,0.000000,0.000000,0.026476,0.000000,0.027479,...,0.04577,0.000000,0.021757,0.0,0.000000,0.000000,0.000000,0.052863,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.013708,0.051403,0.000000,0.0,0.000000,0.011318,0.000000,0.022687,0.010789,0.007849,...,0.00000,0.010895,0.012428,0.0,0.000000,0.000000,0.000000,0.007549,0.000000,0.000000
49996,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.135261,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.0,0.087587,0.000000,0.000000,0.045010,0.000000,0.000000
49997,0.000000,0.000000,0.079427,0.0,0.000000,0.000000,0.000000,0.051777,0.000000,0.053737,...,0.00000,0.074594,0.000000,0.0,0.000000,0.066797,0.000000,0.000000,0.101435,0.000000
49998,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
print("Vacabolary size %.f" % vocab.__len__())

Vacabolary size 192


# Point-wise Mutual Information between words

***

For the PMI we run first the sliding window over the document and count the windows for occurencies of a word and word pairs. 
Next we calculate the relative frequency $p(i)$ and $p(i,j)$ and finaly get the PMI score.


In [7]:
# Filter the docs by by all word including in the vocab (tfidf applies a stop word remover)
names = vocab # vocab from tf-idf calculation
docs_filtered = list(map(lambda x: " ".join([w for w in x.split() if w in names]), docs)) 

# Dictionaries and lookups
word_index = OrderedDict( (name, index) for index, name in enumerate(names) )
index_word = OrderedDict( (idx, name )  for idx, name in enumerate(names) ) 

word_word_index = OrderedDict()
# index_word_word = OrderedDict()

for w1, w2 in tqdm(combinations(names, 2), total = nCr(len(names), 2)):
    word_word = "{}_{}".format(w1,w2)
    word_word_index[word_word] = len(word_word_index)
#     index_word_word[len(word_word_index)] = word_word
    

# This function help later to access the word word dictionary because for w1_w2 = w2_w1 is not both represented
def get_word_word_index(w1, w2):
    word_word = "{}_{}".format(w1, w2)
    idx = word_word_index.get(word_word, "?")
    
    if  idx == "?":
        word_word = "{}_{}".format(w2, w1)
        idx = word_word_index.get(word_word, "?")
    
    return idx


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18336/18336 [00:00<00:00, 162698.14it/s]


In [8]:
# Find the co-occurrences:
window_size = 10 # sliding window size to calculate point-wise mutual information between words

# Window counter
W = 0 # total number of sliding windows in the corpus
W_ij = np.zeros(len(word_word_index), dtype = np.int32) # OrderedDict((ww, 0) for ww in word_word_index.items()) # co-occurencies of word i and word j
W_i  = OrderedDict((name, 0) for name in names) # occurencies of word i


for doc in tqdm(docs_filtered, total = len(docs_filtered)):
    
    # In each doc run a sliding window with size 10
    doc = doc.split()
    for idx in range(len(doc) - window_size):
        W += 1 # count total windows
        
        words = set(doc[idx:(idx + window_size)]) # distict list of words in the current window
    
        # count windows conaining word i
        for word in words:
            W_i[word] += 1
        
        # count windows containing co-occurrences of words i and j 
        for i in combinations(words, 2):
            w1 = i[0]
            w2 = i[1]
            idx = get_word_word_index(w1, w2)
            W_ij[idx] += 1 # Update window counter


# Now can we calculate the probabilities and the PMI score.
            
# Relative frequency
p_ij = pd.Series(W_ij, index = word_word_index.keys()) / W
p_i = pd.Series(W_i, index = W_i.keys()) / W
pmi_ij = p_ij.copy() 

# Calculate the PMI
for w in tqdm(combinations(names, 2), total = nCr(len(names), 2)):
    i = w[0]
    j = w[1] 
    word_word = "{}_{}".format(i, j)
    
    try:
        frac = p_ij[word_word] / (p_i[i] * p_i[j])
        frac = frac + 1E-9 
        pmi = math.log(frac)
        pmi_ij[word_word] = round(pmi, 4)
    except:
        print("ERROR with: ", word_word, frac)
     

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18336/18336 [00:00<00:00, 24222.72it/s]


# Create a Graph

In [9]:
import networkx as nx

def word_word_edges(pmi_ij):
    word_word_edges_list = []
    for w in tqdm(combinations(names, 2), total = nCr(len(names), 2)):
        i = w[0]
        j = w[1] 
        word_word = "{}_{}".format(i, j)
        pmi = pmi_ij.loc[word_word]
        if (pmi > 0):
            word_word_edges_list.append((i, j,{"weight": pmi}))
            
    return word_word_edges_list

# Build graph with document nodes and word nodes
G = nx.Graph()
G.add_nodes_from(df_tfidf.index) # document nodes as index
G.add_nodes_from(vocab) # word nodes

# Build edges between document-word pairs
document_word = [(doc, w, {"weight" : df_tfidf.loc[doc,w]}) for doc in df_tfidf.index for w in df_tfidf.columns if df_tfidf.loc[doc,w] > 0]
G.add_edges_from(document_word)

# Build edges between word-word pairs
word_word = word_word_edges(pmi_ij)
G.add_edges_from(word_word)

save_as_pickle("text_graph.pkl", G)
save_as_pickle("word_word.pkl", word_word)
save_as_pickle("pmi.pkl", pmi_ij)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18336/18336 [00:00<00:00, 23876.12it/s]


.\data\text_graph.pkl
.\data\word_word.pkl
.\data\pmi.pkl


In [ ]:
# Load graph from disk
G = load_pickle("text_graph.pkl")
word_word = load_pickle("word_word.pkl")
pmi_ij = load_pickle("pmi.pkl")

# Semi-Supervised Node Classification
***

Our model $f(X, A)$ is a two layer convolution network with sprectral convolutions. The model is flexible in terms of it relies just on the feature matrix $X = I$ and the adjacancy matrix $A$ of the graph structure.  The feature matrix $X$ is set as an identity matrix $I$, which simply means every word or document is represented as a one-hot vector.
Regarding the author, using the adjacancy matrix is efficient in situations when $X$ possess to less information. 


**Two layer GCN:**

First build a Adjacency and Degree

$$
\hat{\mathbf{A}} = \mathbf{D}^{1/2} \mathbf{A} \mathbf{D}^{1/2}
$$

and the layerwise linear model takes the form

$$
Z = f(X, A) = \text{softmax} \left( \hat{A} \text{ ReLu}(\hat{A}X W_0) W_1  \right)
$$  

where $W_0 \in \mathbb{R}^{C \times H}$ and $W_1 \in \mathbb{R}^{H \times F}$ are the network weights which are trained using gradient descent.
$X$ is a one-hot encoding of each of the graph nodes. 

For the training of a semi-supervised classifier the authors using full dataset batch gradient descent and evaluate the cross-entropy just error over the labeled data: 

$$
L = - \sum_{l \in \mathbb{y}_L} \sum_{f = 1}^{F} Y_{lf} ln Z_{lf}
$$,

with $\mathbb{y}_L$ as set of document idices that have labels and $F$ is the dimension of the output features, which is equal to the number of classes.



<img src = GCN.png width = 800>


**Build Adjacency and Degree**

In [ ]:
# Adjacancy matrix
A = nx.to_numpy_matrix(G, weight = "weight")
A = A + np.eye(G.number_of_nodes()) # diag = 1

# Degree matrix
degrees = []
for d in G.degree(weight = None):
    if d == 0:
        degrees.append(0)
    else:
        degrees.append(d[1]**(-0.5))
degrees = np.diag(degrees)

# A hat
A_hat = degrees@A@degrees \

# Feature matrix
X = np.eye(G.number_of_nodes()) # Features are just identity matrix
f = X # input of the net

save_as_pickle("f.pkl", f)
save_as_pickle("A_hat.pkl", A_hat)


In [ ]:
f = load_pickle("f.pkl")
A_hat = load_pickle("A_hat.pkl")

## Train 

For the trainig we need to tell the net work which nodes in the output layer (word - doc - embedding) is a labeled document node. As we have an semi supervised issue, some of the document embeddings have no label. The output matrix will have the shape of all nodes (words and documents) times the number of classes. Whereas the document indecies are the first. Hence, we just have to select the regarding output by the labeld node-indexes.

In [ ]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow.contrib.slim import fully_connected

# Network Parameters
n_input = f.shape[0] # Input each node in the graph as one-hot encoding
n_classes = 2
dropout = 0.75
n_hidden_1 = 330 # Size of first GCN hidden weights
n_hidden_2 = 130
learning_rate = 0.01

# Graph inputs
X = tf.placeholder(tf.float32, [n_input, n_input])
y = tf.placeholder(tf.float32,  [None, n_classes])

keep_prob  = tf.placeholder(tf.float32)
idx_selected = tf.placeholder(tf.int32, [None])

A_hat_tf = tf.convert_to_tensor(A_hat, dtype = tf.float32)
A_hat_tf = tf.Variable(A_hat_tf)


weights = {
    'h1' : tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2' : tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
}


biases = {
    'b1' : tf.Variable(tf.random_normal([n_hidden_1])),
    'b2' : tf.Variable(tf.random_normal([n_hidden_2]))
}


def convLayer(X, A_hat_tf, w, b ):
    X = tf.add(tf.matmul(X, w), b)  # [?,440][440, 330] + [330] = 440x330
    X = tf.matmul(A_hat_tf, X)  # [440, 440][440,330] = [440,330]
    return X


def gcn(X, weights, biases, A_hat, dropout):

    # First convolution layer ?x330
    conv1 = convLayer(X, A_hat_tf, weights['h1'], biases["b1"])
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.dropout(conv1)
    # Second convolution layer 330x130
    conv2 = convLayer(conv1, A_hat_tf, weights['h2'], biases["b2"])
    # Fully connected layer / Linear layer for logit
    logits = fully_connected(conv2, n_classes, activation_fn = None)
    pred = tf.nn.softmax(logits)
    
    return pred
    

# Build the GCN
pred = gcn(X, weights, biases, A_hat_tf, dropout)

# Filter training document nodes for semi-supervised learning
pred = tf.gather(pred, indices = idx_selected)

# Define optimizer and loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

# Evaluate model
arg_max = [tf.argmax(pred, axis = 1),  tf.argmax(y, axis = 1)]
correct_pred = tf.equal(tf.argmax(pred, axis = 1), tf.argmax(y, axis = 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# Start trainings session
init  = tf.global_variables_initializer()
saver = tf.train.Saver()

logs = {}
logs["acc"] = []
logs["loss"] = []


with tf.Session() as sess:
    
    sess.run(init)
    
    for e in range(5):
        
        batch_x = f # one hot for each node (word + docs) in the graph
        batch_y = np.eye(n_classes)[y_train]
        
        sess.run(optimizer, feed_dict = {X: batch_x,
                                         y: batch_y,
                                         keep_prob: dropout,
                                         idx_selected: idx_train})
        
        loss, acc = sess.run([cost,  accuracy], feed_dict = {X: batch_x,
                                                            y: batch_y,
                                                            keep_prob: 1.,
                                                            idx_selected: idx_train})
        
        print("Epoch ", e, "Batch size: ", batch_y.shape[0] ,"Batch loss: ", loss, "Training accuracy: ", acc)
        
        logs["acc"].append(acc)
        logs["loss"].append(loss)

    
    # save_path = saver.save(sess, "drive/My Drive/Coding/GCN/data/model.ckpt")

    print("Finish!")

    print("Store model weights and biases")
    weights_dict = {}
    for key, values in weights.items():
        weights_dict[key] = sess.run(values)
    save_as_pickle("weights.pkl", weights_dict)

    biases_dict = {}
    for key, values in biases.items():
        biases_dict[key] = sess.run(values)
    save_as_pickle("biases.pkl", biases_dict)

    # Calculate acc for test docs
    batch_x = f
    batch_y = np.eye(n_classes)[y_test]
    test_acc = sess.run(accuracy, feed_dict = {X: batch_x,
                                               y: batch_y,
                                               keep_prob: 1.,
                                               idx_selected: idx_test})
    print("Testing accuracy: ", test_acc, "Batch size: ", batch_y.shape[0])

    sess.close()